# 203: Exampville Destination Choice

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [ ]:
# TEST
import warnings

from pytest import approx

import larch as lx

warnings.filterwarnings(action="ignore", category=FutureWarning)

In [ ]:
import numpy as np
import pandas as pd

import larch as lx
from larch import P, X

In this example notebook, we will walk through the estimation of a tour 
destination choice model.  First, let's load the data files from
our example.

In [ ]:
hh, pp, tour, skims, emp = lx.example(200, ["hh", "pp", "tour", "skims", "emp"])

In [ ]:
hh["INCOME_GRP"] = pd.qcut(hh.INCOME, 3)

For this destination choice model, we'll want to use the mode choice
logsums we calculated previously from the mode choice estimation,
but we'll use these values as fixed input data instead of a modeled value.
We can load these logsums from the file in which they were saved. 
For this example, we can indentify that file using the `larch.example` 
function, which will automatically rebuild the file if it doesn't exists.
In typical applications, a user would generally just give the filename 
as a string and ensure manually that the file exists before loading it.

In [ ]:
logsums_file = lx.example(202, output_file="logsums.zarr")
logsums = lx.DataArray.from_zarr("logsums.zarr")

## Preprocessing

The alternatives in
the destinations model are much more regular than in the mode choice 
model, as every observation will have a similar set of alternatives
and the utility function for each of those alternatives will share a 
common functional form.  We'll leverage this by using `idca` format 
arrays in our DataTree to make data management simpler.  

The base array we'll start with will have two dimensions, cases and
alternatives, anc be formed from the logsums we loaded above.

In [ ]:
ca = lx.Dataset.construct(
    {"logsum": logsums},
    caseid="TOURID",
    alts=skims.TAZ_ID,
)
ca

For our destination choice model, we'll also want to use employment data.
This data, as included in our example, has unique 
values only by alternative and not by caseid, so there are only
40 unique rows.
(This kind of structure is common for destination choice models.)

In [ ]:
emp.info()

Then we bundle all our raw data into a `DataTree` structure, 
which is used to collect the right data for estimation.  The
Larch DataTree is a slightly augmented version of the regular
`sharrow.DataTree`.

In [ ]:
tree = lx.DataTree(
    base=ca,
    tour=tour.rename_axis(index="TOUR_ID"),
    hh=hh.set_index("HHID"),
    person=pp.set_index("PERSONID"),
    emp=emp,
    skims=lx.Dataset.construct.from_omx(skims),
    relationships=(
        "base.TAZ_ID @ emp.TAZ",
        "base.TOURID -> tour.TOUR_ID",
        "tour.HHID @ hh.HHID",
        "tour.PERSONID @ person.PERSONID",
        "hh.HOMETAZ @ skims.otaz",
        "base.TAZ_ID @ skims.dtaz",
    ),
)

## Model Definition

Now we can define our choice model, using data from the tree as appropriate.

In [ ]:
m = lx.Model(datatree=tree)
m.title = "Exampville Work Tour Destination Choice v1"

In [ ]:
m.quantity_ca = (
    +P.EmpRetail_HighInc * X("RETAIL_EMP * (INCOME>50000)")
    + P.EmpNonRetail_HighInc * X("NONRETAIL_EMP") * X("INCOME>50000")
    + P.EmpRetail_LowInc * X("RETAIL_EMP") * X("INCOME<=50000")
    + P.EmpNonRetail_LowInc * X("NONRETAIL_EMP") * X("INCOME<=50000")
)

m.quantity_scale = P.Theta

In [ ]:
m.utility_ca = +P.logsum * X.logsum + P.distance * X.AUTO_DIST

In [ ]:
m.choice_co_code = "tour.DTAZ"

In [ ]:
m.plock(EmpRetail_HighInc=0, EmpRetail_LowInc=0)

In [ ]:
mj = m.copy()

## Model Estimation

In [ ]:
m.compute_engine = "numba"

In [ ]:
m.loglike()

In [ ]:
# TEST
assert m.loglike() == approx(-28238.336880999712)
assert mj.loglike() == approx(-28238.336880999712)

In [ ]:
m.d_loglike()

In [ ]:
mj.d_loglike()

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert result.loglike == approx(-25157.72676137825)
assert result.success
assert result.method == "slsqp"
assert result.n_cases == 7564
# assert result.iteration_number == 12
assert result.logloss == approx(3.325981856342973)
import pandas as pd

pd.testing.assert_series_equal(
    result.x.sort_index(),
    pd.Series(
        {
            "EmpNonRetail_HighInc": 1.366494780535922,
            "EmpNonRetail_LowInc": -0.8786830269346876,
            "EmpRetail_HighInc": 0.0,
            "EmpRetail_LowInc": 0.0,
            "Theta": 0.7489479210277367,
            "distance": -0.041624297118688966,
            "logsum": 1.021914910506737,
        }
    ).sort_index(),
    rtol=2e-2,
)
assert m.pstderr == approx(
    np.array([0.256844, 0.079214, 0.0, 0.0, 0.015231, 0.010723, 0.031744]),
    rel=2e-2,
)

In [ ]:
resultj = mj.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert resultj.loglike == approx(-25157.72676137825)
assert resultj.success
assert resultj.n_cases == 7564
# assert result.iteration_number == 12
assert resultj.logloss == approx(3.325981856342973)
import pandas as pd

pd.testing.assert_series_equal(
    pd.Series(resultj.x, index=mj.pnames).sort_index(),
    pd.Series(
        {
            "EmpNonRetail_HighInc": 1.3639677867685673,
            "EmpNonRetail_LowInc": -0.8813930080108792,
            "EmpRetail_HighInc": 0.0,
            "EmpRetail_LowInc": 0.0,
            "Theta": 0.7493698789904475,
            "distance": -0.04182074282571401,
            "logsum": 1.0208113241683572,
        }
    ).sort_index(),
    rtol=2e-2,
)
assert resultj.stderr == approx(
    np.array(
        [0.25549, 0.079105, 0.0, -0.0, 0.015231, 0.010724, 0.031738], dtype=np.float32
    ),
    rel=1e-2,
)

## Model Visualization

For destination choice and similar type models, it might be beneficial to
review the observed and modeled choices, and the relative distribution of
these choices across different factors.  For example, we would probably want
to see the distribution of travel distance.  The `Model` object includes
a built-in method to create this kind of visualization.

In [ ]:
m.histogram_on_idca_variable("AUTO_DIST")

In [ ]:
m.histogram_on_idca_variable("RETAIL_EMP")

The `histogram_on_idca_variable` has a variety of options,
for example to control the number and range of the histogram bins:

In [ ]:
m.histogram_on_idca_variable("AUTO_DIST", bins=40, span=(0, 10))

Subsets of the observations can be pulled out, to observe the 
distribution conditional on other `idco` factors, like income.

In [ ]:
m.histogram_on_idca_variable(
    "AUTO_DIST",
    x_label="Distance (miles)",
    bins=26,
    span=(0, 13),
    filter_co="INCOME<10000",
)

## Save and Report Model

In [ ]:
report = lx.Reporter(title=m.title)

In [ ]:
report << "# Parameter Summary" << m.parameter_summary()

In [ ]:
report << "# Parameter Summary" << m.parameter_summary()

In [ ]:
report << "# Estimation Statistics" << m.estimation_statistics()

In [ ]:
report << "# Utility Functions" << m.utility_functions()

The figures shown above can also be inserted directly into reports.

In [ ]:
figure = m.histogram_on_idca_variable(
    "AUTO_DIST",
    bins=30,
    span=(0, 15),
    x_label="Distance (miles)",
)
report << "# Visualization"
report << figure

In [ ]:
report.save(
    "exampville_dest_choice.html",
    overwrite=True,
    metadata=m.dumps(),
)